Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce
from random import choice

In [2]:
# function that prints sets in a human readable way
def print_solution(solution, sets):
    
    for i, is_set_covered in enumerate(solution):
        if is_set_covered:
            print('Set', i+1, ': ', end='')
            for element_taken in sets.toarray()[i]:
                if(element_taken): print('x', end='')
                else: print('-', end='')
            print()


In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [4]:
# x = make_set_covering_problem(1000, 1000, .3)
# print("Element at row=42 and column=42:", x[42, 42])

In [5]:
x1 = make_set_covering_problem(100, 100, .3)

In [6]:
x2 = make_set_covering_problem(1000, 1000, .3)

In [22]:
x3 = make_set_covering_problem(5000, 5000, .3)

In [8]:
x4 = make_set_covering_problem(100, 100, .7)

In [9]:
x5 = make_set_covering_problem(1000, 1000, .7)

In [23]:
x6 = make_set_covering_problem(5000, 5000, .7)

In [11]:
def goal_test(sets, state):
    SETS = sets.toarray()
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(SETS[0]))]),
        )
    )

In [12]:
def fitness1(state, sets):
    SETS = sets.toarray()
    valid = np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(len(SETS[0]))]),
        )
    )
    return valid, sum(state)

In [13]:
def tweak1(state, PROBLEM_SIZE, valid):
    new_state = copy(state)
    if valid:
        taken = []
        for i, t in enumerate(state):
            if t:
                taken.append(i)
        index = choice(taken)
    else: index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [14]:
def single_state_solve(initial_state, tweak, fitness, sets, max_improvement=100,total_steps=1_000):
    cycles_needed = 0
    current_state = copy(initial_state)
    valid, current_fitness = fitness(current_state, sets)
    number_of_sets = len(initial_state)
    prev_valid = valid
    failed_improvement = 0
    
    for _ in range(total_steps):
        if failed_improvement >= max_improvement:
            break
        cycles_needed += 1
        new_state = tweak(current_state, number_of_sets, valid)
        valid, new_fitness = fitness(new_state, sets)
        if valid and new_fitness < current_fitness:
            print(f"Improvement at cycle: {cycles_needed}, ({valid}, {new_fitness})") 
            current_state = new_state
            current_fitness = new_fitness
        elif valid and not prev_valid:
            print(f"Found a valid solution at cycle: {cycles_needed}, ({valid}, {new_fitness})")
            current_state = new_state
            current_fitness = new_fitness
        elif not valid and prev_valid:
            failed_improvement += 1
            valid = prev_valid
            continue
        elif not valid and not prev_valid and new_fitness > current_fitness:
            print(f"Still searching a valid solution. Current size {new_fitness}")
            current_state = new_state
            current_fitness = new_fitness
        prev_valid = valid

    return current_state, cycles_needed

In [34]:
initial_state_100 = [False for _ in range(100)]
initial_state_1000 = [False for _ in range(1000)]
initial_state_5000 = [False for _ in range(5000)]

max_improvement = 10

In [35]:

solution1, cycles = single_state_solve(initial_state_100, tweak1, fitness1, x1, max_improvement, 1000)
print(f"Solution has fitness: {fitness1(solution1, x1)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Still searching a valid solution. Current size 8
Still searching a valid solution. Current size 9
Still searching a valid solution. Current size 10
Still searching a valid solution. Current size 11
Still searching a valid solution. Current size 12
Still searching a valid solution. Current size 13
Found a valid solution at cycle: 16, (True, 14)
Improvement at cycle: 18, (True, 13)
Improvement at cycle: 20, (True, 12)
Improvement at cycle: 21, (True, 11)
Improvement at cycle: 23, (True, 10)
Improvement at cycle: 24, (True, 9)
Solution has fitness: (True, 9) and was found in 31 cycles.


In [36]:
solution2, cycles = single_state_solve(initial_state_1000, tweak1, fitness1, x2, max_improvement, 1000)
print(f"Solution has fitness: {fitness1(solution2, x2)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Still searching a valid solution. Current size 8
Still searching a valid solution. Current size 9
Still searching a valid solution. Current size 10
Still searching a valid solution. Current size 11
Still searching a valid solution. Current size 12
Still searching a valid solution. Current size 13
Still searching a valid solution. Current size 14
Still searching a valid solution. Current size 15
Still searching a valid solution. Current size 16
Still searching a valid solution. Current size 17
Still searching a valid solution. Current size 18
Found a valid solution at cycle: 19, (True, 19)
Improvement at cycle: 21, (True, 18)
Improvement at cycle: 23

In [37]:
solution3, cycles = single_state_solve(initial_state_5000, tweak1, fitness1, x3, max_improvement, 1000)
print(f"Solution has fitness: {fitness1(solution3, x3)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Still searching a valid solution. Current size 8
Still searching a valid solution. Current size 9
Still searching a valid solution. Current size 10
Still searching a valid solution. Current size 11
Still searching a valid solution. Current size 12
Still searching a valid solution. Current size 13
Still searching a valid solution. Current size 14
Still searching a valid solution. Current size 15
Still searching a valid solution. Current size 16
Still searching a valid solution. Current size 17
Still searching a valid solution. Current size 18
Still searching a valid solution. Current size 19
Still searching a valid solution. Current size 20
Still sea

In [38]:
solution4, cycles = single_state_solve(initial_state_100, tweak1, fitness1, x4, max_improvement, 1000)
print(f"Solution has fitness: {fitness1(solution4, x4)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Found a valid solution at cycle: 4, (True, 4)
Improvement at cycle: 12, (True, 3)
Solution has fitness: (True, 3) and was found in 15 cycles.


In [39]:
solution5, cycles = single_state_solve(initial_state_1000, tweak1, fitness1, x5, max_improvement, 1000)
print(f"Solution has fitness: {fitness1(solution5, x5)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Found a valid solution at cycle: 6, (True, 6)
Solution has fitness: (True, 6) and was found in 16 cycles.


In [40]:
solution6, cycles = single_state_solve(initial_state_5000, tweak1, fitness1, x6, max_improvement, 1000)
print(f"Solution has fitness: {fitness1(solution6, x6)} and was found in {cycles} cycles.")

Still searching a valid solution. Current size 1
Still searching a valid solution. Current size 2
Still searching a valid solution. Current size 3
Still searching a valid solution. Current size 4
Still searching a valid solution. Current size 5
Still searching a valid solution. Current size 6
Still searching a valid solution. Current size 7
Found a valid solution at cycle: 8, (True, 8)
Improvement at cycle: 11, (True, 7)
Solution has fitness: (True, 7) and was found in 19 cycles.
